In [18]:
from tensorflow import keras
import pandas as pd
from sklearn import preprocessing

In [19]:
df = pd.read_hdf('data_ttl.h5', 'df')

In [20]:
# COMMAND ----------

Y = df[["time_min", "time_hr"]]
#Y = df[["time_hr"]]


X = df[['booktime', 'depdate', 'deptime', 'dur', 'bookdate',
       'fno', 'htg', 'fare', 'posNeg', 'htgGroup']]
del df

print(X.shape, Y.shape)

(9837177, 10) (9837177, 2)


In [21]:
dfH = pd.read_csv("s3://search-curated-sec/HolidayCalendar.csv")

dfH["date"] =dfH.searchdate.str.replace("-","").astype(int)
dfH.drop(columns=["searchdate"], inplace=True)

In [22]:
dfH.columns

Index(['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'weekend', 'Holiday',
       'republicDay', 'Holi', 'GoodFriday', 'RamNavami', 'HoliEve',
       'IdUlFiter', 'Independance', 'IdUlJuha', 'GandhiJayanti', 'Diwali',
       'DiwaliEve', 'Christmas', 'ChristmasEve', 'NewYear', 'MakarSakranti',
       'MahaShivRatri', 'RakshaBandhan', 'Dussehra', 'MahaSaptami',
       'MahaAstami', 'MahaNavami', 'NarakChaturthi', 'BhaiDooj', 'ChattPuja',
       'date'],
      dtype='object')

In [23]:
holiday_df = dfH[["date", "ChattPuja", "weekend", "Holiday"]]

In [24]:
 X = pd.merge(X, holiday_df, how='left', left_on = 'depdate', right_on = 'date')

In [25]:
X.shape

(9837177, 14)

In [26]:
le = preprocessing.LabelEncoder()
X["fno"] =le.fit_transform(X.fno)

In [27]:
X.fillna(0, inplace=True)

In [28]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33)

#del X
#del Y
print("shapes", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

shapes (6590908, 14) (3246269, 14) (6590908, 2) (3246269, 2)


In [29]:
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train)


# COMMAND ----------

X_train =pd.DataFrame(scaler.transform(X_train),
                          columns = X_train.columns,
                          index = X_train.index)

# COMMAND ----------

X_test =pd.DataFrame(scaler.transform(X_test),
                          columns = X_test.columns,
                          index = X_test.index)

In [ ]:
X_train.head()

In [29]:
# COMMAND ----------

dnn = keras.Sequential()

# COMMAND ----------

dnn.add(keras.layers.Dense(X_train.shape[1]/2, input_shape= (X_train.shape[1],), activation="tanh"))
dnn.output_shape

# COMMAND ----------

# dnn.add(keras.layers.Dense(64, activation="relu"))
# dnn.output_shape

# COMMAND ----------

dnn.add(keras.layers.Dropout(0.3))
dnn.output_shape

# COMMAND ----------

#dnn.add(keras.layers.Dense(128, activation="relu"))
#dnn.output_shape

# COMMAND ----------

dnn.add(keras.layers.Dense(y_train.shape[1], activation="relu"))
dnn.output_shape

(None, 1)

In [30]:
optimizer = keras.optimizers.Adamax(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
dnn.compile(optimizer=optimizer, loss="mean_squared_error", metrics=["accuracy"],)
#dnn.compile(loss='mse', # keras.losses.mean_squared_error
#            optimizer=keras.optimizers.SGD(lr=0.2),
#            metrics=["accuracy"])

In [ ]:
dnn.fit(x=X_train, 
        y= y_train, 
        validation_data=(X_test[:500], y_test[:500]), 
        batch_size=300,
        epochs=100,
        shuffle=False
       )

Train on 6590908 samples, validate on 500 samples
Epoch 1/100
6590908/6590908 [==============================] - 150s 23us/sample - loss: 91860.9121 - acc: 0.0017 - val_loss: 55946.4320 - val_acc: 0.0020
Epoch 2/100
6590908/6590908 [==============================] - 159s 24us/sample - loss: 91102.0271 - acc: 0.0017 - val_loss: 56027.5406 - val_acc: 0.0020
Epoch 3/100
6590908/6590908 [==============================] - 122s 18us/sample - loss: 90893.4691 - acc: 0.0017 - val_loss: 56078.4688 - val_acc: 0.0020
Epoch 4/100
5827500/6590908 [=========================>....] - ETA: 15s - loss: 90695.3144 - acc: 0.0017

## Random Forest

In [30]:
from sklearn.ensemble import RandomForestRegressor

In [31]:
X_train.head()

,booktime,depdate,deptime,dur,bookdate,fno,htg,fare,posNeg,htgGroup,date,ChattPuja,weekend,Holiday
7632224,0.058537,0.033865,0.728051,0.192580,0.058538,0.577075,0.013575,0.162694,1.0,0.055556,0.999483,0.0,0.0,0.0
450233,0.057336,0.029903,0.394004,0.194346,0.057327,0.499012,0.012182,0.101863,0.0,0.055556,0.999479,0.0,0.0,0.0
9029299,0.049036,0.025507,0.314775,0.199647,0.049031,0.979249,0.007077,0.155776,1.0,0.055556,0.999475,0.0,0.0,0.0
8937307,0.074868,0.054541,0.081370,0.197880,0.074851,0.010870,0.325792,0.101928,1.0,0.333333,0.999505,0.0,1.0,1.0
6911030,0.067588,0.048406,0.740899,0.164311,0.067580,0.458498,0.193294,0.236538,1.0,0.222222,0.999498,0.0,1.0,1.0


In [34]:
regressor = RandomForestRegressor( n_estimators=15)
regressor.fit(X_train, y_train)

MemoryError: could not allocate 469762048 bytes

In [ ]:
regressor.score(X_test, y_test)

In [15]:
regressor = RandomForestRegressor( n_estimators=20)
regressor.fit(X_train, y_train)

/root/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=20,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [16]:
regressor.score(X_test, y_test)

0.9802556787513825